# $\color{#19ABEA}{\text{Run GPTQ and full precision LLM models in Latest Version TextGen-webui :}}$

In [ ]:
#@title # $\color{#19ABEA}{\text{Download and install requirements}}$ {display-mode: "form"}
#@markdown $\color{#19ABEA}{\text{run this only once}}$

#@markdown `Note :no need to check these checkboxes unless u facing a problem`
#only if u want to remuve existing folder#!rm -rf /content/text-generation-webui
%cd /content
!git clone -b snapshot-2023-12-24 https://github.com/oobabooga/text-generation-webui
# !git clone -b V20231127 https://github.com/Troyanovsky/text-generation-webui
%cd /content/text-generation-webui
!pip install -r requirements.txt
!pip install git+https://github.com/UWUplus/flask-cloudflared# the origin
!pip install -r extensions/openai/requirements.txt --upgrade
!apt-get -y install -qq aria2

reinstall_llama_cpp_python = False # @param {type:"boolean"}
if reinstall_llama_cpp_python :
  !pip uninstall -y llama-cpp-python
  !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir

#@markdown  `reinstalling flash attentionv2 may be needed  for some models like phi2..`
reinstall_flash_attention = False # @param {type:"boolean"}
if reinstall_flash_attention:
  !pip uninstall -y flash-attn
  !pip install --no-build-isolation flash-attn==2.3.0

from IPython.display import clear_output
clear_output()
print("Done")

In [ ]:
#@title #  $\color{#19ABEA}{Download Model}$ {display-mode: "form"}
#@markdown ## $\color{#19ABEA}{\text{Copy the RepoName here }}$
#@markdown you can use Past GPTQ model or fullPrecision here such as
#@markdown   :`teknium/OpenHermes-2.5-Mistral-7B`
#@markdown  or :`TheBloke/xDAN-L1-Chat-RL-v1-GPTQ` etc...
repo_name = "TheBloke/xDAN-L1-Chat-RL-v1-GPTQ" # @param ["TheBloke/Mistral-7B-OpenOrca-GPTQ", "TheBloke/zephyr-7B-alpha-GPTQ", "TheBloke/zephyr-7B-beta-GPTQ", "TheBloke/neural-chat-7B-v3-1-GPTQ", "TheBloke/mistral-ft-optimized-1218-GPTQ", "TheBloke/xDAN-L1-Chat-RL-v1-GPTQ", "TheBloke/openchat-3.5-1210-starling-slerp-GPTQ", "TheBloke/Loyal-Macaroni-Maid-7B-GPTQ", "TheBloke/Valkyrie-V1-GPTQ", "TheBloke/Sonya-7B-GPTQ", "TheBloke/Starling-LM-7B-alpha-GPTQ", "TheBloke/SOLAR-10.7B-v1.0-GPTQ"] {allow-input: true}
# repo_name = "xDAN-AI/xDAN-L1-Chat-RL-v1" # @param ["teknium/OpenHermes-2.5-Mistral-7B", "xDAN-AI/xDAN-L1-Chat-RL-v1"]  { allow-input: true}

#@markdown  in colab gpu (15Gvram) you can use :
#@markdown - `13b model GPTQ`
#@markdown - `7b model GPTQ`
#@markdown - you can use full precision 13b , 7b modles but using flags like `--load-in-4bit` or `--load-in-8bit`  its a bit slower then quantized versions but works well
repo_link =f"https://huggingface.co/{repo_name}"
print(repo_link)
%cd /content/text-generation-webui/models
!git lfs install
!git clone $repo_link
print("Done")

In [ ]:
#@title ##  $\color{#19ABEA}{RunOnly}$ {display-mode: "form"}
# %cd /content/text-generation-webui
#@markdown  $\color{#19ABEA}{\text{Enter ur Context size or let it Unckechet to use the default size}}$
params =" "
#@markdown _____
api =True # @param {type:"boolean"}
if api :params +=" --api --public-api --listen "
api_only_mode =False # @param {type:"boolean"}
if api_only_mode : params+=" --nowebui "
#@markdown _____
use_custom_context_len = False # @param {type:"boolean"}
context_len=4096*4 #@param
if use_custom_context_len : params += f" --n_ctx  {context_len} "
#@markdown _____
use_load_in_flag = False # @param {type:"boolean"}
load_in = "--load-in-8bit" # @param ["--load-in-8bit","--load-in-4bit"] {type:"string"}
if use_load_in_flag: params +=f" {load_in} "
#@markdown _____
use_custom_params = False # @param {type:"boolean"}
custom_params = "" # @param {type:"string"}
if use_custom_params : params +=custom_params


#@markdown _____
# params
%cd /content/text-generation-webui
model_file_name =repo_name.split('/')[-1]
!python server.py   --share  --n-gpu-layers 100000  	--model $model_file_name   $params